### XGBoost 모델의 매개변수 탐색을 위한 Hyperopt을 활용한 파라미터 최적화

### 학습 내용
 * 데이터 준비 및 전처리
 * XGBoost 모델 학습 및 예측
 * Hyperopt을 사용한 파라미터 탐색

* Hyperopt은 파라미터 최적화를 위한 파이썬 라이브러리입니다. 주어진 하이퍼파라미터 공간에서 가장 우수한 조합을 찾는데 사용됩니다. Hyperopt은 베이지안 최적화 알고리즘 중 하나인 트리-파라미터(TPE)를 구현한 tpe 알고리즘을 제공

### 사전 준비 및 코드 실행 환경
 * xgboost, hyperopt 설치가 되어 있어야 함.
 * 설치 후, 라이브러리 버전 확인(23/06)
    * xgboost 버전 : 1.7.5
    * hyperopt 버전 : 0.2.7

### 데이터 준비 및 전처리

In [1]:
# ---------------------------------
# 데이터 등의 사전 준비
# ----------------------------------
import numpy as np
import pandas as pd

# train_x는 학습 데이터, train_y는 목적 변수, test_x는 테스트 데이터
# pandas의 DataFrame, Series로 유지합니다.(numpy의 array로 유지하기도 합니다)

train = pd.read_csv('../input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x = pd.read_csv('../input/sample-data/test_preprocessed.csv')

In [2]:
train.shape, train_x.shape, train_y.shape, test_x.shape

((10000, 29), (10000, 28), (10000,), (10000, 28))

### 데이터 나누기 및 XGB 모델 클래스 생성

In [3]:
# 학습 데이터를 학습 데이터와 검증 데이터로 나눔
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

# xgboost로 학습 및 예측을 수행
import xgboost as xgb


class Model:

    def __init__(self, params=None):
        self.model = None
        if params is None:
            self.params = {}
        else:
            self.params = params

    def fit(self, tr_x, tr_y, va_x, va_y):
        #params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
        params = {'objective': 'binary:logistic', 'verbosity': 0, 'random_state': 71}
        params.update(self.params)
        num_round = 10
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        dvalid = xgb.DMatrix(va_x, label=va_y)
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        self.model = xgb.train(params, dtrain, num_round, evals=watchlist)

    def predict(self, x):
        data = xgb.DMatrix(x)
        pred = self.model.predict(data)
        return pred

In [4]:
# -----------------------------------
# 탐색할 파라미터의 공간의 지정
# -----------------------------------
# hp.choice에서는 복수의 선택사항에서 고르기
# hp.uniform에서는 하한상한을 지정한 동일분포로부터 추출. 인수는 하한상한
# hp.quniform에서는 하한상한을 지정한 균등분포 중 일정 간격마다의 점으로부터 추출. 인수는 하한상한간격
# hp.loguniform에서는 하한상한을 지정한 로그을 취한 값이 균등분포를 따르는 분포로부터 추출. 인수는 하한상한의 로그를 취한 값

# 만약 moduleNotFoundError 발생시 hyperopt 설치 후 진행
# 2023/06/16 hyperopt 설치 및 실행 확인 : 0.2.7
# pip install hyperopt
from hyperopt import hp

space = {
    'activation': hp.choice('activation', ['prelu', 'relu']),
    'dropout': hp.uniform('dropout', 0, 0.2),
    'units': hp.quniform('units', 32, 256, 32),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.00001), np.log(0.01)),
}

### 모델 학습 

In [8]:
# 탐색할 매개변수의 공간을 지정
space = {
    'min_child_weight': hp.quniform('min_child_weight', 1, 5, 1),
    'max_depth': hp.quniform('max_depth', 3, 9, 1),
    'gamma': hp.quniform('gamma', 0, 0.4, 0.1),
}

# hyperopt에 의한 매개변수 탐색 실행
max_evals = 10
trials = Trials()
history = []
fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

[0]	train-logloss:0.54916	eval-logloss:0.55556                                                                         
[1]	train-logloss:0.46704	eval-logloss:0.47825                                                                         
[2]	train-logloss:0.41289	eval-logloss:0.42694                                                                         
[3]	train-logloss:0.37646	eval-logloss:0.39516                                                                         
[4]	train-logloss:0.34842	eval-logloss:0.37219                                                                         
[5]	train-logloss:0.32789	eval-logloss:0.35661                                                                         
[6]	train-logloss:0.30931	eval-logloss:0.34235                                                                         
[7]	train-logloss:0.29330	eval-logloss:0.32852                                                                         
[8]	train-logloss:0.28254	eval-logloss:0

[2]	train-logloss:0.37612	eval-logloss:0.40783                                                                         
[3]	train-logloss:0.33334	eval-logloss:0.37238                                                                         
[4]	train-logloss:0.29993	eval-logloss:0.34467                                                                         
[5]	train-logloss:0.27425	eval-logloss:0.32465                                                                         
[6]	train-logloss:0.25230	eval-logloss:0.30979                                                                         
[7]	train-logloss:0.23468	eval-logloss:0.30111                                                                         
[8]	train-logloss:0.21924	eval-logloss:0.29117                                                                         
[9]	train-logloss:0.20572	eval-logloss:0.28550                                                                         
params: {'gamma': 0.4, 'max_depth': 8, '

{'gamma': 0.4, 'max_depth': 8.0, 'min_child_weight': 1.0}

In [9]:
# 기록한 정보에서 매개변수와 점수를 출력
# (trials에서도 정보를 취득할 수 있지만 매개변수의 취득이 다소 어려움)
history = sorted(history, key=lambda tpl: tpl[1])
best = history[0]
print(f'best params:{best[0]}, score:{best[1]:.4f}')

best params:{'gamma': 0.4, 'max_depth': 8, 'min_child_weight': 1.0}, score:0.2805
